# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#########################################################################
import matplotlib.font_manager as fm
import matplotlib as mpl
#########################################################################
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
#########################################################################
from tqdm import tqdm

# 1. 한글 폰트 경로 설정 (Windows 기본: 맑은 고딕)
font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = fm.FontProperties(fname=font_path).get_name()

# 2. matplotlib에 폰트 적용
mpl.rc("font", family=font_name)

# 3. 마이너스 기호 깨짐 방지
mpl.rcParams["axes.unicode_minus"] = False

In [ ]:
jemu_df = pd.read_csv('../../data/재무정보_final_v2(16~24).csv', encoding='utf-8-sig')
jemu_df.tail(1)

# 전체 데이터 eda

## DF 형태 파악

In [ ]:
# 고유 기업명만
jemu_df["기업명"].unique()

In [ ]:
# 특정 기업에 대한 행 보기
jemu_df[jemu_df["기업명"] == "휴비오 주식회사"]

### 컬럼명들

In [ ]:
jemu_df.columns

In [ ]:
jemu_df.info()

### 기업별 행 개수
- 모든 값이 다 6이 나와야 정상 -> ok

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명").size().sort_values(ascending=False)

### 기업별 결측치 비율
- 1에 가까울수록 거의 다 결측

In [ ]:
company_missing_ratio = (
    jemu_df
    .groupby("기업명")
    .apply(lambda x: x.isna().mean())
)

company_missing_ratio.head()

## 변수 기준으로 기업별 분포 보기
- (우선) 변수 기준 : 매출액
-> 이렇게 하면 빈 값 확인 가능

In [ ]:
jemu_df.columns = jemu_df.columns.str.strip()

In [ ]:
jemu_df.columns.tolist()

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명")['매출액'].count().sort_values()   # 기업별 매출액에 값 들어있는 개수

### 매출액

In [ ]:
sales_count = (
    jemu_df
    .groupby("기업명")['매출액']
    .count()
)

count_distribution = sales_count.value_counts().sort_index()

plt.figure()
bars =plt.bar(
    count_distribution.index,
    count_distribution.values
)
plt.xticks(count_distribution.index)
plt.xlabel("기업별 매출액 값 개수 (연도 수)")
plt.ylabel("기업 수")
plt.title("매출액 데이터 보유 개수별 기업 분포")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        int(height),
        ha="center",
        va="bottom"
    )

plt.show()

### 연구개발비

In [ ]:
sales_count = (
    jemu_df
    .groupby("기업명")["연구개발비"]
    .count()
)

count_distribution = sales_count.value_counts().sort_index()

plt.figure()
bars =plt.bar(
    count_distribution.index,
    count_distribution.values
)
plt.xticks(count_distribution.index)
plt.xlabel("기업별 연구개발비 값 개수 (연도 수)")
plt.ylabel("기업 수")
plt.title("연구개발비 데이터 보유 개수별 기업 분포")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        int(height),
        ha="center",
        va="bottom"
    )

plt.show()

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기

ales_count = (
    jemu_df
    .groupby("기업명")["매출액"]
    .count()
)

zero_sales_companies = sales_count[sales_count == 6]

zero_sales_companies.index.tolist()

# KNN-test
        1) 같은 연도 + 같은 산업(sectors/field) 후보 추출
        2) 이웃 기준 컬럼으로 KNN 이웃 K개 선택
        3) 그 이웃을 공통으로 사용
        4) 8개 재무 컬럼을 성격별 규칙으로 대체
        5) KNN 실패 시 Cold deck fallback

### sectors 기준

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# 0. 기본 설정
# =====================================================
FINANCIAL_COLS = [
    '매출액', '영업이익', '당기순이익',
    '자산총계', '부채총계', '자본총계',
    '연구개발비', 'CAPEX'
]

COMPANY_COL = '기업명'
YEAR_COL = '연도'
INDUSTRY_COL = 'sectors'


# =====================================================
# 1. 숫자형 변환 (기본 전처리)
# =====================================================
def ensure_numeric(df):
    df = df.copy()
    df[FINANCIAL_COLS] = df[FINANCIAL_COLS].apply(
        pd.to_numeric, errors='coerce'
    )
    return df


# =====================================================
# 2. 기업 내부 시계열 보간 (index-safe)
# =====================================================
def impute_by_company_timeseries(df):
    df = df.sort_values([COMPANY_COL, YEAR_COL]).copy()

    for col in FINANCIAL_COLS:
        df[col] = (
            df.groupby(COMPANY_COL)[col]
            .transform(lambda x: x.interpolate(limit=1))
        )

    return df


# =====================================================
# 3. 유사 기업 중앙값 대체 (연도 왜곡 방지)
# =====================================================
def impute_by_similar_companies_safe(df):
    df_imputed = df.copy()

    for idx, row in df.iterrows():
        if not row[FINANCIAL_COLS].isnull().any():
            continue

        company = row[COMPANY_COL]
        year = row[YEAR_COL]
        industry = row[INDUSTRY_COL]

        # 같은 산업 + 같은 연도
        candidates = df[
            (df[INDUSTRY_COL] == industry) &
            (df[YEAR_COL] == year) &
            (df[COMPANY_COL] != company)
        ]

        # 표본 수가 충분할 때만
        if len(candidates) < 5:
            continue

        for col in FINANCIAL_COLS:
            if pd.isna(row[col]):
                valid_vals = candidates[col].dropna()
                if len(valid_vals) >= 5:
                    df_imputed.at[idx, col] = valid_vals.median()

    return df_imputed


# =====================================================
# 4. KNN Imputation (최후 수단, shape-safe)
# =====================================================
def knn_impute_safe(df, n_neighbors=5):
    df_imputed = df.copy()

    for (industry, year), group in df.groupby([INDUSTRY_COL, YEAR_COL]):

        # 실제 값이 존재하는 컬럼만
        valid_cols = [
            c for c in FINANCIAL_COLS
            if group[c].notna().sum() > 0
        ]

        # 최소 조건
        if len(group) <= n_neighbors or len(valid_cols) <= 1:
            continue

        imputer = KNNImputer(
            n_neighbors=min(n_neighbors, len(group) - 1),
            weights='distance'
        )

        imputed_array = imputer.fit_transform(group[valid_cols])

        # ndarray → DataFrame (index / columns 명시)
        imputed_df = pd.DataFrame(
            imputed_array,
            index=group.index,
            columns=valid_cols
        )

        df_imputed.loc[group.index, valid_cols] = imputed_df

    return df_imputed


# =====================================================
# 5. 전체 파이프라인
# =====================================================
def final_imputation_pipeline(df):
    df = ensure_numeric(df)

    print("▶ Step 1: 기업 내부 시계열 보간")
    df = impute_by_company_timeseries(df)

    print("▶ Step 2: 유사 기업 중앙값 대체 (보수적)")
    df = impute_by_similar_companies_safe(df)

    print("▶ Step 3: KNN 대체 (최후 수단)")
    df = knn_impute_safe(df)

    return df


# =====================================================
# 6. 실행
# =====================================================
df_final = final_imputation_pipeline(jemu_df)

print("\n[최종 변수별 남은 결측치]")
print(df_final[FINANCIAL_COLS].isnull().sum())


In [ ]:
df_final.to_csv("../../output/jemu_df_imputed_sectors_ver5.csv", index=False, encoding="cp949")

### fields 기준

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics.pairwise import nan_euclidean_distances
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 0단계: CSV 로드 + 컬럼명 정리
# ========================================
jemu_df = pd.read_csv(
    '../../data/재무정보_final_v2(16~24).csv',
    encoding='utf-8-sig'
)

# 🔴 핵심: 컬럼명 앞뒤 공백 제거
jemu_df.columns = jemu_df.columns.str.strip()


# ========================================
# 공통 설정
# ========================================
FINANCIAL_COLS = [
    '매출액', '영업이익', '당기순이익',
    '자산총계', '부채총계', '자본총계',
    '연구개발비', 'CAPEX'
]

COMPANY_COL = '기업명'
YEAR_COL = '연도'
INDUSTRY_COL = 'field'


# ========================================
# 1단계: 재무 컬럼 숫자형 강제 변환
# ========================================
def ensure_numeric_financial_cols(df):
    df = df.copy()
    df[FINANCIAL_COLS] = df[FINANCIAL_COLS].apply(
        pd.to_numeric, errors='coerce'
    )
    return df


# ========================================
# 2단계: 기업 프로필 생성
# ========================================
def create_company_profile(df):
    company_profile = (
        df.groupby(COMPANY_COL)
        .agg({
            '매출액': 'median',
            '자산총계': 'median',
            INDUSTRY_COL: 'first'
        })
        .reset_index()
    )

    company_profile['규모'] = pd.cut(
        company_profile['매출액'],
        bins=[0, 100, 500, 1000, np.inf],
        labels=['소형', '중소형', '중형', '대형']
    )
    company_profile['규모'] = company_profile['규모'].cat.add_categories(['Unknown'])
    company_profile.loc[company_profile['매출액'].isnull(), '규모'] = 'Unknown'

    return company_profile


# ========================================
# 3단계: 중앙값 대체 + 추적
# ========================================
def impute_by_similar_companies(df, company_profile):
    df_imputed = df.copy()

    # 🔹 추적 컬럼 생성
    df_imputed['impute_method'] = np.nan
    df_imputed['impute_source_companies'] = np.nan
    df_imputed['impute_k'] = np.nan

    missing_rows = df[df[FINANCIAL_COLS].isnull().any(axis=1)]

    for idx, row in missing_rows.iterrows():
        company = row[COMPANY_COL]
        year = row[YEAR_COL]

        profile_row = company_profile[company_profile[COMPANY_COL] == company]
        if profile_row.empty:
            continue
        profile = profile_row.iloc[0]

        similar_companies = []

        # 1순위: 산업 + 규모
        if profile['규모'] != 'Unknown':
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile['규모'] == profile['규모']) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        # 2순위: 산업만
        if len(similar_companies) < 3:
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        similar_data = df[
            (df[COMPANY_COL].isin(similar_companies)) &
            (df[YEAR_COL] == year)
        ]

        used_companies = similar_data[
            similar_data[FINANCIAL_COLS].notnull().any(axis=1)
        ][COMPANY_COL].unique()

        for col in FINANCIAL_COLS:
            if pd.isnull(row[col]) and similar_data[col].dropna().shape[0] >= 3:
                df_imputed.at[idx, col] = similar_data[col].median()
                df_imputed.at[idx, 'impute_method'] = 'median_group'
                df_imputed.at[idx, 'impute_source_companies'] = ', '.join(used_companies)

    return df_imputed


# ========================================
# 4단계: KNN Imputation + 추적
# ========================================
def knn_impute_by_industry_year(df, n_neighbors=5):
    df_imputed = df.copy()

    for (industry, year), group in df.groupby([INDUSTRY_COL, YEAR_COL]):
        idxs = group.index
        data = group[FINANCIAL_COLS]

        if len(group) <= n_neighbors:
            continue

        for idx in idxs:
            row = data.loc[idx]
            if not row.isnull().any():
                continue

            valid = data.dropna()
            if len(valid) < 2:
                continue

            distances = nan_euclidean_distances(
                row.values.reshape(1, -1),
                valid.values
            )[0]

            k = min(n_neighbors, len(valid))
            nearest_idx = valid.index[np.argsort(distances)[:k]]
            source_companies = df.loc[nearest_idx, COMPANY_COL].tolist()

            for col in FINANCIAL_COLS:
                if pd.isnull(df_imputed.at[idx, col]):
                    df_imputed.at[idx, col] = valid.loc[nearest_idx, col].mean()
                    df_imputed.at[idx, 'impute_method'] = 'knn'
                    df_imputed.at[idx, 'impute_source_companies'] = ', '.join(source_companies)
                    df_imputed.at[idx, 'impute_k'] = k

    return df_imputed


# ========================================
# 5단계: 통합 실행
# ========================================
def hybrid_imputation(df):
    df_result = ensure_numeric_financial_cols(df)

    print("=" * 50)
    print("결측치 처리 시작")
    print("=" * 50)
    print(f"초기 결측치: {df_result[FINANCIAL_COLS].isnull().sum().sum()}개\n")

    company_profile = create_company_profile(df_result)

    print("[Step 1] 중앙값 기반 보완 중...")
    df_result = impute_by_similar_companies(df_result, company_profile)
    remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
    print(f"✓ 남은 결측치: {remaining}개")

    if remaining > 0:
        print("[Step 2] KNN 기반 보완 중...")
        df_result = knn_impute_by_industry_year(df_result)
        remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
        print(f"✓ 남은 결측치: {remaining}개")

    print("결측치 처리 완료")
    print("=" * 50)

    return df_result, company_profile


# ========================================
# 6단계: 실행
# ========================================
df_final, company_profile = hybrid_imputation(jemu_df)

print("\n[변수별 남은 결측치]")
print(df_final[FINANCIAL_COLS].isnull().sum())


In [ ]:
df_final.to_csv("../../output/jemu_df_imputed_fields_ver2.csv", index=False, encoding="cp949")